In [2]:
SEED = 42
DATA_PATH = "data/"

In [5]:
import pandas as pd

train = pd.read_csv(f"{DATA_PATH}loan_grade_train.csv")
train.shape

(19548, 10)

In [6]:
train.head()

,ID,나이,연간소득,주택소유상태,근로기간,대출목적,대출금액,이자율,신용거래기간,target
0,train_0,26,33996,임대,10.0,투자,7000,8.00,2,0
1,train_1,34,84000,임대,3.0,개인사업,9000,5.79,10,0
2,train_2,22,24702,임대,NaN,교육,2000,13.23,4,2
3,train_3,24,69996,임대,3.0,부채통합,2500,12.53,3,1
4,train_4,26,58125,모기지론,10.0,투자,6000,10.25,2,1


```
근로기간과 이자율만 결측치가 생길 수 있다고 가정
```

In [7]:
train.isnull().sum()

ID           0
나이           0
연간소득         0
주택소유상태       0
근로기간       571
대출목적         0
대출금액         0
이자율       1874
신용거래기간       0
target       0
dtype: int64

In [10]:
train["근로기간"].mean(), train["이자율"].mean()

(4.810612847130737, 10.998815774584136)

In [29]:
def preprocessing_data(df, cat_enc=None, scaler=None):
    cols = [
        "나이",
        "연간소득",
        "주택소유상태",
        "근로기간",
        "대출목적",
        "대출금액",
        "이자율",
        "신용거래기간",
    ]
    df = df[cols].copy()

    df["근로기간"] = df["근로기간"].fillna(4.810612847130737)
    df["이자율"] = df["이자율"].fillna(10.998815774584136)

    from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

    cols = df.select_dtypes("object").columns
    if cat_enc is None:
        cat_enc = OneHotEncoder(handle_unknown="ignore")
        cat_enc.fit(df[cols])

    tmp = cat_enc.transform(df[cols]).toarray()
    df[cat_enc.get_feature_names_out()] = tmp
    df = df.drop(columns=cols)

    if scaler is None:
        scaler = MinMaxScaler()
        scaler.fit(df)

    return scaler.transform(df), cat_enc, scaler

In [31]:
train_ft, cat_enc, scaler = preprocessing_data(train)

In [32]:
test = pd.read_csv(f"{DATA_PATH}loan_grade_test.csv")

In [33]:
test_ft, _, _ = preprocessing_data(test, cat_enc, scaler)

In [35]:
test.iloc[:1]

,ID,나이,연간소득,주택소유상태,근로기간,대출목적,대출금액,이자율,신용거래기간
0,test_0,25,36000,임대,2.0,개인사업,10000,NaN,2


In [36]:
test_data, _, _ = preprocessing_data(test.iloc[:1], cat_enc, scaler)
test_data

array([[0.04032258, 0.01571876, 0.01626016, 0.27536232, 0.33426098,
        0.        , 0.        , 0.        , 0.        , 1.        ,
        1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])